# Setup
(No need to read)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-nnyr33xb
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-nnyr33xb
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit f5a7d455546a88cfdfb26e781d5bd6447e8243eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 2.8 MB/s eta 0:00:00
    

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [3]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [4]:
model = HookedTransformer.from_pretrained("gpt-neo-2.7B")

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Loaded pretrained model gpt-neo-2.7B into HookedTransformer


# Fibonacci

In [5]:
example_prompt = "0, 1, 1, 2, 3, 5,"
example_answer = " 8"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ',', ' 1', ',', ' 1', ',', ' 2', ',', ' 3', ',', ' 5', ',']
Tokenized answer: [' 8']


Performance on answer token:
Rank: 3        Logit: 16.34 Prob: 12.20% Token: | 8|

Top 0th token. Logit: 17.20 Prob: 28.76% Token: | 7|
Top 1th token. Logit: 16.59 Prob: 15.70% Token: | 10|
Top 2th token. Logit: 16.41 Prob: 13.07% Token: | 6|
Top 3th token. Logit: 16.34 Prob: 12.20% Token: | 8|
Top 4th token. Logit: 15.42 Prob:  4.85% Token: | 5|
Top 5th token. Logit: 15.37 Prob:  4.60% Token: | 9|
Top 6th token. Logit: 14.83 Prob:  2.70% Token: | 4|
Top 7th token. Logit: 14.52 Prob:  1.98% Token: | 0|
Top 8th token. Logit: 14.41 Prob:  1.77% Token: | 11|
Top 9th token. Logit: 14.37 Prob:  1.70% Token: | 1|


Ranks of the answer tokens: [(' 8', 3)]

In [6]:
example_prompt = "0 1 1 2 3 5"
example_answer = " 8"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ' 1', ' 1', ' 2', ' 3', ' 5']
Tokenized answer: [' 8']


Performance on answer token:
Rank: 2        Logit: 15.35 Prob: 13.54% Token: | 8|

Top 0th token. Logit: 16.07 Prob: 27.79% Token: | 6|
Top 1th token. Logit: 15.42 Prob: 14.61% Token: | 7|
Top 2th token. Logit: 15.35 Prob: 13.54% Token: | 8|
Top 3th token. Logit: 15.22 Prob: 11.86% Token: | 10|
Top 4th token. Logit: 14.24 Prob:  4.46% Token: | 4|
Top 5th token. Logit: 14.21 Prob:  4.35% Token: |
|
Top 6th token. Logit: 14.15 Prob:  4.07% Token: | 5|
Top 7th token. Logit: 13.88 Prob:  3.11% Token: | 9|
Top 8th token. Logit: 13.42 Prob:  1.97% Token: | 0|
Top 9th token. Logit: 12.91 Prob:  1.19% Token: | 11|


Ranks of the answer tokens: [(' 8', 2)]

In [7]:
example_prompt = "0 1 1 2 3 5 8 13 21"
example_answer = " 34"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ' 1', ' 1', ' 2', ' 3', ' 5', ' 8', ' 13', ' 21']
Tokenized answer: [' 34']


Performance on answer token:
Rank: 0        Logit: 18.04 Prob: 25.31% Token: | 34|

Top 0th token. Logit: 18.04 Prob: 25.31% Token: | 34|
Top 1th token. Logit: 17.92 Prob: 22.43% Token: | 31|
Top 2th token. Logit: 17.34 Prob: 12.53% Token: | 29|
Top 3th token. Logit: 17.12 Prob: 10.10% Token: | 28|
Top 4th token. Logit: 16.45 Prob:  5.16% Token: | 32|
Top 5th token. Logit: 16.37 Prob:  4.77% Token: | 33|
Top 6th token. Logit: 16.11 Prob:  3.68% Token: |
|
Top 7th token. Logit: 16.02 Prob:  3.35% Token: | 30|
Top 8th token. Logit: 15.76 Prob:  2.59% Token: | 26|
Top 9th token. Logit: 15.08 Prob:  1.31% Token: | 27|


Ranks of the answer tokens: [(' 34', 0)]

# 2 4 6 8

In [8]:
example_prompt = "2 4 6 8 10 12"
example_answer = " 14"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 6', ' 8', ' 10', ' 12']
Tokenized answer: [' 14']


Performance on answer token:
Rank: 0        Logit: 17.61 Prob: 87.34% Token: | 14|

Top 0th token. Logit: 17.61 Prob: 87.34% Token: | 14|
Top 1th token. Logit: 13.57 Prob:  1.54% Token: | 15|
Top 2th token. Logit: 13.43 Prob:  1.34% Token: | 16|
Top 3th token. Logit: 13.25 Prob:  1.11% Token: | 13|
Top 4th token. Logit: 13.20 Prob:  1.06% Token: |
|
Top 5th token. Logit: 12.08 Prob:  0.35% Token: |14|
Top 6th token. Logit: 11.88 Prob:  0.28% Token: | -|
Top 7th token. Logit: 11.88 Prob:  0.28% Token: | 18|
Top 8th token. Logit: 11.81 Prob:  0.26% Token: | 1|
Top 9th token. Logit: 11.79 Prob:  0.26% Token: |]|


Ranks of the answer tokens: [(' 14', 0)]

In [9]:
example_prompt = "2 4 8 16 32 64"
example_answer = " 128"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 8', ' 16', ' 32', ' 64']
Tokenized answer: [' 128']


Performance on answer token:
Rank: 0        Logit: 16.56 Prob: 48.11% Token: | 128|

Top 0th token. Logit: 16.56 Prob: 48.11% Token: | 128|
Top 1th token. Logit: 14.98 Prob:  9.94% Token: |
|
Top 2th token. Logit: 14.51 Prob:  6.19% Token: | 256|
Top 3th token. Logit: 13.81 Prob:  3.07% Token: | 80|
Top 4th token. Logit: 13.59 Prob:  2.47% Token: | 96|
Top 5th token. Logit: 13.52 Prob:  2.31% Token: | 1|
Top 6th token. Logit: 13.36 Prob:  1.96% Token: | 1024|
Top 7th token. Logit: 13.35 Prob:  1.94% Token: | 64|
Top 8th token. Logit: 13.01 Prob:  1.38% Token: | 32|
Top 9th token. Logit: 12.73 Prob:  1.05% Token: | -|


Ranks of the answer tokens: [(' 128', 0)]

In [10]:
example_prompt = "4 8 16 32 64"
example_answer = " 128"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '4', ' 8', ' 16', ' 32', ' 64']
Tokenized answer: [' 128']


Performance on answer token:
Rank: 0        Logit: 15.87 Prob: 34.87% Token: | 128|

Top 0th token. Logit: 15.87 Prob: 34.87% Token: | 128|
Top 1th token. Logit: 14.53 Prob:  9.19% Token: |
|
Top 2th token. Logit: 14.07 Prob:  5.78% Token: | 80|
Top 3th token. Logit: 13.96 Prob:  5.15% Token: | 64|
Top 4th token. Logit: 13.94 Prob:  5.08% Token: | 256|
Top 5th token. Logit: 13.85 Prob:  4.62% Token: | 96|
Top 6th token. Logit: 13.59 Prob:  3.56% Token: | 1|
Top 7th token. Logit: 13.50 Prob:  3.26% Token: | 32|
Top 8th token. Logit: 12.40 Prob:  1.09% Token: | |
Top 9th token. Logit: 12.24 Prob:  0.93% Token: | 1024|


Ranks of the answer tokens: [(' 128', 0)]

In [11]:
" ".join([str(2**i) for i in range(5, 5+7)])

'32 64 128 256 512 1024 2048'

In [12]:
example_prompt = " ".join([str(2**i) for i in range(5, 5+7)])
example_answer = " 4096"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '32', ' 64', ' 128', ' 256', ' 512', ' 1024', ' 2048']
Tokenized answer: [' 4096']


Performance on answer token:
Rank: 1        Logit: 15.22 Prob: 16.77% Token: | 4096|

Top 0th token. Logit: 15.34 Prob: 18.99% Token: |
|
Top 1th token. Logit: 15.22 Prob: 16.77% Token: | 4096|
Top 2th token. Logit: 14.80 Prob: 11.04% Token: | 30|
Top 3th token. Logit: 14.47 Prob:  7.98% Token: |

|
Top 4th token. Logit: 13.99 Prob:  4.94% Token: | |
Top 5th token. Logit: 13.42 Prob:  2.79% Token: | 5|
|
Top 7th token. Logit: 12.87 Prob:  1.60% Token: | 8|
Top 8th token. Logit: 12.83 Prob:  1.54% Token: | 40|
Top 9th token. Logit: 12.76 Prob:  1.43% Token: | 25|


Ranks of the answer tokens: [(' 4096', 1)]

# arithm plus 1

In [13]:
example_prompt = "1 + 1 ="
example_answer = " 2"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' +', ' 1', ' =']
Tokenized answer: [' 2']


Performance on answer token:
Rank: 1        Logit: 14.71 Prob: 16.84% Token: | 2|

Top 0th token. Logit: 15.27 Prob: 29.26% Token: | 0|
Top 1th token. Logit: 14.71 Prob: 16.84% Token: | 2|
Top 2th token. Logit: 13.95 Prob:  7.85% Token: | 1|
Top 3th token. Logit: 13.36 Prob:  4.33% Token: | 3|
Top 4th token. Logit: 13.20 Prob:  3.70% Token: | -|
Top 5th token. Logit: 12.92 Prob:  2.79% Token: | 4|
Top 6th token. Logit: 12.61 Prob:  2.07% Token: | \|
Top 7th token. Logit: 12.47 Prob:  1.79% Token: | 5|
Top 8th token. Logit: 12.39 Prob:  1.65% Token: | n|
Top 9th token. Logit: 12.14 Prob:  1.28% Token: | m|


Ranks of the answer tokens: [(' 2', 1)]

In [14]:
example_prompt = "5 + 1 ="
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '5', ' +', ' 1', ' =']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 18       Logit: 14.40 Prob:  0.84% Token: | 6|

Top 0th token. Logit: 17.90 Prob: 27.80% Token: | 0|
Top 1th token. Logit: 17.70 Prob: 22.81% Token: | -|
Top 2th token. Logit: 16.71 Prob:  8.49% Token: | 2|
Top 3th token. Logit: 16.20 Prob:  5.11% Token: | 3|
Top 4th token. Logit: 15.82 Prob:  3.48% Token: | 4|
Top 5th token. Logit: 15.74 Prob:  3.20% Token: | 5|
Top 6th token. Logit: 15.01 Prob:  1.54% Token: | 1|
Top 7th token. Logit: 14.83 Prob:  1.30% Token: | n|
Top 8th token. Logit: 14.78 Prob:  1.23% Token: | s|
Top 9th token. Logit: 14.76 Prob:  1.21% Token: | d|


Ranks of the answer tokens: [(' 6', 18)]

In [15]:
example_prompt = "10 + 1 ="
example_answer = " 11"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '10', ' +', ' 1', ' =']
Tokenized answer: [' 11']


Performance on answer token:
Rank: 31       Logit: 13.40 Prob:  0.59% Token: | 11|

Top 0th token. Logit: 16.89 Prob: 19.37% Token: | -|
Top 1th token. Logit: 16.70 Prob: 16.02% Token: | 0|
Top 2th token. Logit: 16.18 Prob:  9.48% Token: | 2|
Top 3th token. Logit: 15.86 Prob:  6.87% Token: | 3|
Top 4th token. Logit: 15.51 Prob:  4.89% Token: | 5|
Top 5th token. Logit: 15.37 Prob:  4.21% Token: | 4|
Top 6th token. Logit: 14.82 Prob:  2.44% Token: | 1|
Top 7th token. Logit: 14.41 Prob:  1.62% Token: | 6|
Top 8th token. Logit: 14.41 Prob:  1.61% Token: | n|
Top 9th token. Logit: 14.25 Prob:  1.38% Token: | d|


Ranks of the answer tokens: [(' 11', 31)]

In [16]:
example_prompt = "1 + 1 = 2. 10 + 1 = 11. 3 + 1 = 4. 5 + 1 ="
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' +', ' 1', ' =', ' 2', '.', ' 10', ' +', ' 1', ' =', ' 11', '.', ' 3', ' +', ' 1', ' =', ' 4', '.', ' 5', ' +', ' 1', ' =']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 20.74 Prob: 82.95% Token: | 6|

Top 0th token. Logit: 20.74 Prob: 82.95% Token: | 6|
Top 1th token. Logit: 18.56 Prob:  9.39% Token: | 7|
Top 2th token. Logit: 17.66 Prob:  3.80% Token: | 8|
Top 3th token. Logit: 16.79 Prob:  1.61% Token: | 9|
Top 4th token. Logit: 15.64 Prob:  0.50% Token: | 10|
Top 5th token. Logit: 14.99 Prob:  0.26% Token: | 11|
Top 6th token. Logit: 14.79 Prob:  0.22% Token: | 5|
Top 7th token. Logit: 14.63 Prob:  0.18% Token: | 1|
Top 8th token. Logit: 14.18 Prob:  0.12% Token: |6|
Top 9th token. Logit: 14.06 Prob:  0.10% Token: | 2|


Ranks of the answer tokens: [(' 6', 0)]

In [17]:
example_prompt = "1 table 2. 10 table 11. 3 table 4. 5 table"
example_answer = " 6"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 10', ' table', ' 11', '.', ' 3', ' table', ' 4', '.', ' 5', ' table']
Tokenized answer: [' 6']


Performance on answer token:
Rank: 0        Logit: 16.10 Prob: 39.79% Token: | 6|

Top 0th token. Logit: 16.10 Prob: 39.79% Token: | 6|
Top 1th token. Logit: 15.52 Prob: 22.24% Token: | 5|
Top 2th token. Logit: 15.01 Prob: 13.31% Token: |
|
Top 3th token. Logit: 13.56 Prob:  3.14% Token: | 1|
Top 4th token. Logit: 13.22 Prob:  2.21% Token: | 7|
Top 5th token. Logit: 12.77 Prob:  1.42% Token: | 2|
Top 6th token. Logit: 12.69 Prob:  1.31% Token: | 12|
Top 7th token. Logit: 12.51 Prob:  1.09% Token: | 8|
Top 8th token. Logit: 12.10 Prob:  0.73% Token: | 10|
Top 9th token. Logit: 12.02 Prob:  0.67% Token: | 11|


Ranks of the answer tokens: [(' 6', 0)]

In [18]:
example_prompt = "1 table 2. 10 table 11. 3 table 4. 15 table"
example_answer = " 16"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 10', ' table', ' 11', '.', ' 3', ' table', ' 4', '.', ' 15', ' table']
Tokenized answer: [' 16']


Performance on answer token:
Rank: 4        Logit: 13.27 Prob:  2.41% Token: | 16|

Top 0th token. Logit: 15.94 Prob: 34.88% Token: | 5|
Top 1th token. Logit: 15.49 Prob: 22.33% Token: | 6|
Top 2th token. Logit: 15.05 Prob: 14.36% Token: |
|
Top 3th token. Logit: 13.54 Prob:  3.16% Token: | 1|
Top 4th token. Logit: 13.27 Prob:  2.41% Token: | 16|
Top 5th token. Logit: 12.97 Prob:  1.79% Token: | 2|
Top 6th token. Logit: 12.92 Prob:  1.70% Token: | 12|
Top 7th token. Logit: 12.49 Prob:  1.11% Token: | 7|
Top 8th token. Logit: 12.48 Prob:  1.10% Token: | 8|
Top 9th token. Logit: 11.90 Prob:  0.61% Token: | 15|


Ranks of the answer tokens: [(' 16', 4)]

In [19]:
example_prompt = "1 table 2. 10 table 11. 3 table 4. 101 table"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 2', '.', ' 10', ' table', ' 11', '.', ' 3', ' table', ' 4', '.', ' 101', ' table']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 7        Logit: 12.07 Prob:  0.74% Token: | 102|

Top 0th token. Logit: 16.46 Prob: 59.30% Token: | 5|
Top 1th token. Logit: 14.89 Prob: 12.35% Token: |
|
Top 2th token. Logit: 13.85 Prob:  4.39% Token: | 6|
Top 3th token. Logit: 13.37 Prob:  2.72% Token: | 2|
Top 4th token. Logit: 12.92 Prob:  1.72% Token: | 12|
Top 5th token. Logit: 12.59 Prob:  1.24% Token: | 1|
Top 6th token. Logit: 12.29 Prob:  0.92% Token: | 101|
Top 7th token. Logit: 12.07 Prob:  0.74% Token: | 102|
Top 8th token. Logit: 11.97 Prob:  0.67% Token: | 10|
Top 9th token. Logit: 11.96 Prob:  0.66% Token: | 3|


Ranks of the answer tokens: [(' 102', 7)]

# in-context greater-than

In [20]:
example_prompt = "1 table 4. 10 table 100. 3 table 6. 101 table"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 4', '.', ' 10', ' table', ' 100', '.', ' 3', ' table', ' 6', '.', ' 101', ' table']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 15       Logit: 11.99 Prob:  1.49% Token: | 102|

Top 0th token. Logit: 13.88 Prob:  9.83% Token: | 2|
Top 1th token. Logit: 13.66 Prob:  7.92% Token: | 100|
Top 2th token. Logit: 13.43 Prob:  6.30% Token: |
|
Top 3th token. Logit: 13.41 Prob:  6.15% Token: | 1|
Top 4th token. Logit: 13.36 Prob:  5.87% Token: | 3|
Top 5th token. Logit: 13.13 Prob:  4.67% Token: | 6|
Top 6th token. Logit: 13.07 Prob:  4.37% Token: | 4|
Top 7th token. Logit: 13.01 Prob:  4.14% Token: | 7|
Top 8th token. Logit: 12.75 Prob:  3.18% Token: | 5|
Top 9th token. Logit: 12.71 Prob:  3.07% Token: |.|


Ranks of the answer tokens: [(' 102', 15)]

In [21]:
example_prompt = "1 table 4. 10 table 100. 3 table 6. 1 table"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 4', '.', ' 10', ' table', ' 100', '.', ' 3', ' table', ' 6', '.', ' 1', ' table']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 52       Logit:  9.58 Prob:  0.12% Token: | 102|

Top 0th token. Logit: 14.09 Prob: 10.78% Token: | 4|
Top 1th token. Logit: 14.05 Prob: 10.39% Token: | 2|
Top 2th token. Logit: 13.97 Prob:  9.55% Token: | 100|
Top 3th token. Logit: 13.78 Prob:  7.88% Token: | 1|
Top 4th token. Logit: 13.53 Prob:  6.18% Token: | 3|
Top 5th token. Logit: 13.50 Prob:  6.00% Token: |
|
Top 6th token. Logit: 13.24 Prob:  4.62% Token: | 6|
Top 7th token. Logit: 13.23 Prob:  4.56% Token: | 5|
Top 8th token. Logit: 13.00 Prob:  3.64% Token: | 7|
Top 9th token. Logit: 12.92 Prob:  3.34% Token: | 10|


Ranks of the answer tokens: [(' 102', 52)]

In [22]:
example_prompt = "1 table 4. 10 table 100. 3 table 6. 4 table"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' table', ' 4', '.', ' 10', ' table', ' 100', '.', ' 3', ' table', ' 6', '.', ' 4', ' table']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 39       Logit: 10.11 Prob:  0.20% Token: | 102|

Top 0th token. Logit: 13.80 Prob:  8.06% Token: | 7|
Top 1th token. Logit: 13.72 Prob:  7.51% Token: | 100|
Top 2th token. Logit: 13.61 Prob:  6.71% Token: | 6|
Top 3th token. Logit: 13.60 Prob:  6.64% Token: | 2|
Top 4th token. Logit: 13.59 Prob:  6.59% Token: | 1|
Top 5th token. Logit: 13.49 Prob:  5.96% Token: | 8|
Top 6th token. Logit: 13.43 Prob:  5.58% Token: | 5|
Top 7th token. Logit: 13.33 Prob:  5.06% Token: | 3|
Top 8th token. Logit: 13.29 Prob:  4.85% Token: | 4|
Top 9th token. Logit: 13.12 Prob:  4.10% Token: | 10|


Ranks of the answer tokens: [(' 102', 39)]